# 경로 설정

In [ ]:
from pathlib import Path
import pandas as pd

# Path 정리
DATA_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/data")
BASE_DIR = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/tojihoo/ASAN_01_mini_sam3/")
CSV_PATH = DATA_DIR / "metadata.csv"

# CSV 불러오기
df = pd.read_csv(CSV_PATH)

target = 1                             # 원하는 행 인덱스 설정
COMMON_PATH = df.loc[target,"common_path"]   # COMMON_PATH 추출

VIDEO_PTH = df.loc[target,"video_path"]
N_FRAMES = df.loc[target,"n_frames"]                    # 프레임 수 추출

FRAME_DIR = DATA_DIR / "1_FRAME" / COMMON_PATH          # 프레임 디렉토리 경로 설정
KPT_DIR = DATA_DIR / "2_KEYPOINTS" / COMMON_PATH        # 키포인트 디렉토리 경로 설정
MP4_PATH = DATA_DIR / "3_MP4" / f"{COMMON_PATH}.mp4"     # MP4 디렉토리 경로 설정    
INTERP_DIR = DATA_DIR / "4_INTERP_DATA" /COMMON_PATH    # 보간 데이터 디렉토리 경로 설정
OUTPUT_PATH = DATA_DIR / "test"

CHECKPOINT_DIR = DATA_DIR / "checkpoints/SAM3"
CHECKPOINT_PT = CHECKPOINT_DIR / "sam3.pt"

In [ ]:
import sys

print("\n" + "="*60)
print("🐞 DEBUGGING REPORT")
print("="*60)

# 1. 기본 경로 및 CSV 확인
print(f"\n[1] 기본 경로 설정 확인")
print(f"   - DATA_DIR: {DATA_DIR} [{'✅ Exists' if DATA_DIR.exists() else '❌ Missing'}]")
print(f"   - BASE_DIR: {BASE_DIR} [{'✅ Exists' if BASE_DIR.exists() else '❌ Missing'}]")
print(f"   - CSV_PATH: {CSV_PATH} [{'✅ Exists' if CSV_PATH.exists() else '❌ Missing'}]")

# 2. 타겟 데이터 정보 (DataFrame 추출 값)
print(f"\n[2] Target Data Info (Index: {target})")
print(f"   - COMMON_PATH: {COMMON_PATH}")
print(f"   - N_FRAMES   : {N_FRAMES} (Type: {type(N_FRAMES).__name__})")
print(f"   - VIDEO_PTH (Raw): {VIDEO_PTH}")

# 3. 주요 디렉토리 및 파일 존재 여부 검증
# VIDEO_PTH는 문자열일 수 있으므로 Path 객체로 변환하여 확인
video_path_obj = Path(VIDEO_PTH)
print(f"\n[3] 주요 경로 유효성 검사")
print(f"   - Video File   : {video_path_obj} [{'✅ Exists' if video_path_obj.exists() else '❌ Missing'}]")
print(f"   - MP4_PATH     : {MP4_PATH} [{'✅ Exists' if MP4_PATH.exists() else '❌ Missing'}]")
print(f"   - FRAME_DIR    : {FRAME_DIR} [{'✅ Exists' if FRAME_DIR.exists() else '❌ Missing'}]")
print(f"   - KPT_DIR      : {KPT_DIR} [{'✅ Exists' if KPT_DIR.exists() else '❌ Missing'}]")
print(f"   - INTERP_DIR   : {INTERP_DIR} [{'✅ Exists' if INTERP_DIR.exists() else '❌ Missing'}]")
print(f"   - OUTPUT_PATH  : {OUTPUT_PATH} [{'✅ Exists' if OUTPUT_PATH.exists() else '⚠️ Will be created'}]")

# 4. 체크포인트 확인
print(f"\n[4] 모델 체크포인트 확인")
print(f"   - Checkpoint Dir : {CHECKPOINT_DIR} [{'✅ Exists' if CHECKPOINT_DIR.exists() else '❌ Missing'}]")
print(f"   - Checkpoint File: {CHECKPOINT_PT} [{'✅ Exists' if CHECKPOINT_PT.exists() else '❌ Missing'}]")

print("\n" + "="*60 + "\n")

# 필수 경로가 없을 경우 경고 발생 (선택 사항)
if not video_path_obj.exists():
    print("🚨 WARNING: 원본 비디오 파일이 존재하지 않습니다!")
if not CHECKPOINT_PT.exists():
    print("🚨 WARNING: 모델 가중치 파일(sam3.pt)이 존재하지 않습니다!")

# kpt에서 Boundary Box 가지고 오기

In [ ]:
import json
import numpy as np

with open(KPT_DIR / "000000.json", 'r') as f:
    kpt_data = json.load(f)

ori_bbox = kpt_data['instance_info'][0]['bbox'][0]
ori_bbox = np.array(ori_bbox,dtype=np.float32)

kpt_width, kpt_height = 1280, 720

rel_box = [[
    ori_bbox[0] / kpt_width,
    ori_bbox[1] / kpt_height,
    ori_bbox[2] / kpt_width,
    ori_bbox[3] / kpt_height
]]
rel_box = np.array(rel_box,dtype=np.float32)

print("Original BBox:", ori_bbox)
print("Relative BBox:", rel_box)

# SAM3 모델 빌드

In [ ]:
from pathlib import Path
from sam3.model_builder import build_sam3_video_model

sam3_model = build_sam3_video_model(checkpoint_path=CHECKPOINT_PT)  # SAM3 비디오 모델 빌드
predictor = sam3_model.tracker                                      # SAM3 비디오 예측기 초기화
predictor.backbone = sam3_model.detector.backbone                   # 백본 설정

print("SAM3 비디오 모델과 예측기가 성공적으로 초기화되었습니다.")

In [ ]:
inference_state = predictor.init_state(video_path=VIDEO_PTH)    # 비디오 추론 상태 초기화
predictor.clear_all_points_in_video(inference_state)            # 비디오의 모든 포인트 지우기

In [ ]:
import cv2

# load the frames for visualization
cap = cv2.VideoCapture(VIDEO_PTH)
video_frames_for_vis = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    video_frames_for_vis.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
cap.release()
frame0 = video_frames_for_vis[0]

width, height = frame0.shape[1], frame0.shape[0]

In [ ]:
import matplotlib.pyplot as plt
from sam3.visualization_utils import show_box, show_mask
ann_frame_idx = 0  # the frame index we interact with
ann_obj_id = 4  # give a unique id to each object we interact with (it can be any integers)


_, out_obj_ids, low_res_masks, video_res_masks  = predictor.add_new_points_or_box(
    inference_state=inference_state,
    frame_idx=ann_frame_idx,
    obj_id=ann_obj_id,
    box=rel_box,
)

box=np.array([[ 
    rel_box[0][0] * width,
    rel_box[0][1] * height,
    rel_box[0][2] * width,
    rel_box[0][3] * height
]], dtype=np.float32)


# show the results on the current (interacted) frame
plt.figure(figsize=(9, 6))
plt.title(f"frame {ann_frame_idx}")
plt.imshow(video_frames_for_vis[ann_frame_idx])
show_box(box[0], plt.gca())
show_mask((video_res_masks[0] > 0.0).cpu().numpy(), plt.gca(), obj_id=ann_obj_id)

In [ ]:
import time

start_time = time.time()
# run propagation throughout the video and collect the results in a dict
video_segments = {}  # video_segments contains the per-frame segmentation results
for frame_idx, obj_ids, low_res_masks, video_res_masks, obj_scores in predictor.propagate_in_video(inference_state, start_frame_idx=0, max_frame_num_to_track=N_FRAMES, reverse=False, propagate_preflight=True):
    video_segments[frame_idx] = {
        out_obj_id: (video_res_masks[i] > 0.0).cpu().numpy()
        for i, out_obj_id in enumerate(out_obj_ids)
    }

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Propagation completed in {elapsed_time:.2f} seconds for {N_FRAMES} frames.")




## 시각화

In [ ]:

# render the segmentation results every few frames
vis_frame_stride = 30
plt.close("all")
for out_frame_idx in range(0, len(video_frames_for_vis), vis_frame_stride):
    plt.figure(figsize=(6, 4))
    plt.title(f"frame {out_frame_idx}")
    plt.imshow(video_frames_for_vis[out_frame_idx])
    for out_obj_id, out_mask in video_segments[out_frame_idx].items():
        show_mask(out_mask, plt.gca(), obj_id=out_obj_id)